## Home assignment 05: Bagging and OOB score

Please, fill the lines in the code below.
This is a simplified version of `BaggingRegressor` from `sklearn`. Please, notice, that `sklearn` API is **not preserved**.

Your algorithm should be able to train different instances of the same model class on bootstrapped datasets and to provide [OOB score](https://en.wikipedia.org/wiki/Out-of-bag_error) for the training set.

The model should be passed as model class with no explicit parameters and no parentheses.

Example:
```
import numpy as np
from sklearn.linear_model import LinearRegression

bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
bagging_regressor.fit(LinearRegression, X, y)

```

In [7]:
import numpy as np


class SimplifiedBaggingRegressor:
    def __init__(self, num_bags, oob=False):
        self.num_bags = num_bags
        self.oob = oob

    def _generate_splits(self, data):
        self.indices_list = []
        data_length = len(data)
        for _ in range(self.num_bags):
            indices = np.random.randint(0, data_length, size=data_length)
            self.indices_list.append(indices)

    def fit(self, model_constructor, data, target):
        self.data = None
        self.target = None
        self._generate_splits(data)

        assert len(set(list(map(len, self.indices_list)))) == 1, 'All bags should be of the same length!'
        assert list(map(len, self.indices_list))[0] == len(data), 'All bags should contain `len(data)` number of elements!'

        self.models_list = []
        for indices in self.indices_list:
            model = model_constructor()
            data_bag = data[indices]
            target_bag = target[indices]
            self.models_list.append(model.fit(data_bag, target_bag))

        if self.oob:
            self.data = data
            self.target = target

    def predict(self, data):
        predictions = np.zeros((self.num_bags, len(data)))
        for i, model in enumerate(self.models_list):
            predictions[i] = model.predict(data)
        return np.mean(predictions, axis=0)

    def _get_oob_predictions_from_every_model(self):
        list_of_predictions_lists = [[] for _ in range(len(self.data))]

        for model_idx, bag_indices in enumerate(self.indices_list):
            # Находим индексы объектов, которые не попали в текущий bag
            mask = np.ones(len(self.data), dtype=bool)
            mask[bag_indices] = False
            oob_indices = np.where(mask)[0]

            if len(oob_indices) > 0:
                predictions = self.models_list[model_idx].predict(self.data[oob_indices])
                for idx, pred in zip(oob_indices, predictions):
                    list_of_predictions_lists[idx].append(pred)

        self.list_of_predictions_lists = np.array(list_of_predictions_lists, dtype=object)

    def _get_averaged_oob_predictions(self):
        self._get_oob_predictions_from_every_model()

        oob_predictions = []
        for predictions in self.list_of_predictions_lists:
            if len(predictions) > 0:
                oob_predictions.append(np.mean(predictions))
            else:
                oob_predictions.append(None)

        self.oob_predictions = np.array(oob_predictions, dtype=float)

    def OOB_score(self):
        self._get_averaged_oob_predictions()

        # Находим объекты с OOB предсказаниями
        mask = ~np.isnan(self.oob_predictions)
        if not np.any(mask):
            return np.nan

        # Считаем MSE только для объектов с OOB предсказаниями
        return np.mean((self.oob_predictions[mask] - self.target[mask]) ** 2)


### Local tests:

In [8]:
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm

#### Simple tests:

In [9]:
for _ in tqdm(range(100)):
    X = np.random.randn(2000, 10)
    y = np.mean(X, axis=1)
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=10, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    assert np.mean((predictions - y)**2) < 1e-6, 'Linear dependency should be fitted with almost zero error!'
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score < 1e-6, 'OOB error for linear dependency should be also close to zero!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'

print('Simple tests done!')

  0%|          | 0/100 [00:00<?, ?it/s]

Simple tests done!


#### Medium tests

In [10]:
for _ in tqdm(range(10)):
    X = np.random.randn(200, 150)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=20, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    average_train_error = np.mean((predictions - y)**2)
    assert bagging_regressor.oob, 'OOB feature must be turned on'
    oob_score = bagging_regressor.OOB_score()
    assert oob_score > average_train_error, 'OOB error must be higher than train error due to overfitting!'
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 0.1, 'Probability of missing a bag should be close to theoretical value!'

print('Medium tests done!')

  0%|          | 0/10 [00:00<?, ?it/s]

Medium tests done!


#### Complex tests:

In [11]:
for _ in tqdm(range(10)):
    X = np.random.randn(2000, 15)
    y = np.random.randn(len(X))
    bagging_regressor = SimplifiedBaggingRegressor(num_bags=100, oob=True)
    bagging_regressor.fit(LinearRegression, X, y)
    predictions = bagging_regressor.predict(X)
    oob_score = bagging_regressor.OOB_score()
    assert abs(
        np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)) < 1e-2, 'Probability of missing a bag should be close to theoretical value!'

print('Complex tests done!')

  0%|          | 0/10 [00:00<?, ?it/s]

Complex tests done!


In [12]:
np.mean(
            list(map(len, bagging_regressor.list_of_predictions_lists))
        ) / bagging_regressor.num_bags - 1/np.exp(1)

-0.00030444117144234895

Great job! Please, save `SimplifiedBaggingRegressor` to  `bagging.py` and submit your solution to the grading system!